In [ ]:
from read_data import *
from hyper import *
from alignment import *
from datasets.preprecossing import *

In [ ]:
datas = pd.read_csv("./datas/data1/data_cell.csv").set_index("Unnamed: 0")
datas

In [ ]:
celltype  = cell_type1 = pd.read_csv("../../../capital/docs/tutorials/BRCA_EMTAB8107_CellMetainfo_table.tsv",sep="\t")
celltype

In [ ]:
adata1 = datas.copy()
adata1.loc[list(celltype['Cell']), 'Celltype'] = list(celltype['Celltype (minor-lineage)'])
adata1

In [ ]:
from scipy.spatial import KDTree
ans_value = []
ans_label = []
true_label = [];
now_labels = adata1['Celltype'].tolist();
c= -71
r= 10
now_label = 0;
now =  datas.values;
while len(now) != 0:
    rnd = np.random.randint(now.shape[0], size=1);
    rand_choice = now[rnd, :].reshape(-1)
    tree = KDTree(now);
    indices = tree.query_ball_point(rand_choice,r)
    points_within_k = now[indices]
    now = now.tolist();
    if(len(points_within_k)-1 < c):
        for i in points_within_k:
            ans_value.append(i.tolist())
            ans_label.append(now_label);
            now_label +=1
            now.remove(i.tolist());
            true_label.append(now_labels[int(rnd)])
    else:
        for i in points_within_k:
            ans_value.append(i.tolist())
            ans_label.append(now_label);
            now.remove(i.tolist());            
            true_label.append(now_labels[int(rnd)])
        now_label+=1;
    now = np.array(now);
    print("remain length: {}".format(len(now)))
    # for i in points_within_k:
    #     if(i == rand_choice).all():
    #         continue;
        
    # now.remove(rand_choice)
    # now.tolist().remove(rand_choice.tolist())
    # now = np.array(now)
    # break
# rand_choice == now
np.save('./datas/try/c_{}_r_{}_label.npy'.format(c,r),np.array(ans_label))
np.save('./datas/try/c_{}_r_{}_value.npy'.format(c,r),np.array(ans_value))
np.save('./datas/try/c_{}_r_{}_true.npy'.format(c,r),np.array(true_label))



In [ ]:
v = pd.DataFrame(ans_value)
v['label'] = ans_label
ann1  = v.groupby("label").mean()

In [ ]:
v1 = pd.DataFrame(true_label)
v1['label'] = ans_label
meta1 = v1.groupby("label").max()

In [ ]:
total = len(v1['label'])
correct = 0;
for i in range(len(v1['label'])):
    # print(v1['label'][i]);
    # print(v1[0][i])
    if(v1[0][i] == meta1[0][v1['label'][i]]):
        correct+=1;
print(correct/total)

In [ ]:
ann1

In [ ]:
ann2 = ann1.copy()
ann2.index = meta1[0]

In [ ]:
ann1.to_csv('./datas/72/datas.data',header=None)
ann1.to_csv('./datas/72/datas.csv')


In [ ]:
x, y_true, similarities = load_data('./datas/72/datas.data',1,514,0)


In [ ]:
x, y_true, similarities = load_data('./datas/72/datas.data',1,514,0)
data_path = './datas/72/datas.data'
start = 1
end =514
lable = 0
model_path = None;
save_path = './datas/72/'
epoches=10
c=-1
np.random.seed(1234)
torch.manual_seed(1234)
# x, y_true, similarities = load_data('../../../cityu/HypHC/data/4_8/4_8.data',start,end,lable)
# x, y_true, similarities = load_data(data_path,start,end,lable)
print("{} length:{}".format(data_path,len(y_true)));
dataset = HCDataset(x, y_true, similarities, num_samples=50000)
dataloader = data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)
if(model_path ==None):
    model = HypHC(dataset.n_nodes, 2, 5e-2, 5e-2 ,0.999)
    model.to("cpu")
    Optimizer = getattr(optim, 'RAdam')
    optimizer = Optimizer(model.parameters(),0.0005)
    train(model,dataloader,optimizer,similarities,epoches);
    torch.save(model,save_path+'model.pth');
else:
    params = torch.load((model_path), map_location=torch.device('cpu'))
    model.load_state_dict(params, strict=False)
model.eval()
sim_fn = lambda x, y: torch.sum(x * y, dim=-1)
n=len(x);
d = model.normalize_embeddings(model.embeddings.weight.data)
d = project(d).detach().cpu()
ijs = sl_np_mst_ij(d,sim_fn)
uf = UnionFind(n,d,c)
uf.merge(ijs)
count=0


In [ ]:
tree = nx.DiGraph()
for i, j in enumerate(uf.tree()[:-1]):
    if(j!=-1):
        tree.add_edge(j, i)
        
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111)
circle = plt.Circle((0, 0), 20.0, color='r', alpha=0.1)
ax.add_artist(circle)

n = len(d)
embeddings = np.array(uf.pos)
# for i in range(len(embeddings)):
#     if(np.isnan(embeddings[i][0]) or embeddings[i][0] + embeddings[i][1] == 0):
#         embeddings[i] = embeddings[i-1];
#     pass;
# print(embeddings)
where_are_NaNs = np.isnan(embeddings)
embeddings[where_are_NaNs] = 0
vis = uf.vis
# xlist=[]
# ylist=[]
# labels=[]

# for i in range(n):
#     if(uf.vis[i]!=1):
#         xlist.append(embeddings[i,0]*20);
#         ylist.append(embeddings[i,1]*20);
#         labels.append(y_true[i]);
# # colors = get_colors(labels, 1234)
colors = get_colors(y_true, 1234)

# ax.scatter(xlist, ylist, c=colors, s=50, alpha=0.6)

ax.scatter(embeddings[:n, 0]*20, embeddings[:n, 1]*20, c=colors, s=50, alpha=0.6)

for i in range(len(embeddings)):
    if(i<n):
        continue;
    if(uf.mer[i]!=-1):
        # print(i,'pass')
        pass;
    else:
        ax.scatter(embeddings[i][0]*20,embeddings[i][1]*20,color='black',s=20,alpha=0.7)

    

for n1, n2 in tree.edges():
#     if(uf.vis[n1]== 1 or uf.vis[n2]== 1 ):
# #         print("1")
#         pass
#     else:
#         x1 = embeddings[n1]
#         x2 = embeddings[n2]
# #         if(n1>=n):
# #             if(uf.mer[n1]!=-1):
# #                 pass;
# #             else:
# #                 count+=1;
# #         if(n2>n):
# #             if(uf.mer[n1]!=-1):
# # # 
# #                 pass;
# #             else:
# # #                 ax.scatter(x2[0]*20,x2[1]*20,color='black',s=20,alpha=0.7)
# #                 count+=1
    
    
#         plot_geodesic(x1, x2, ax)
# #         print(x1,x2,n1,n2)
# #         break
#         pass

#     ax.axis("off")
    x1 = embeddings[n1];
    x2 = embeddings[n2];
    plot_geodesic(x1,x2,ax)

In [ ]:
from alignment import node
from alignment import show_tree
embeddings = np.array(uf.pos)


In [ ]:
nodes1 = [node(name=str(i),son=[]) for i in range(len(uf.tree()))]
for i,j in enumerate(uf.tree()):
    if(j!=-1):
        nodes1[j].son.append(nodes1[i])
    nodes1[i].value=embeddings[i];
    nodes1[i].subson=0
    # if(i==49):
    #     break


In [ ]:
def search_tree(now,c):
    
    if(len(now.son) != 2):
        return now;
    lson = search_tree(now.son[0],c);
    now.son[0] = lson;
    rson = search_tree(now.son[1],c);
    now.son[1] = rson
    # lson = now.son[0]
    # rson = now.son[1]
    if(np.linalg.norm(lson.value-rson.value)<=c):
        if(len(lson.son)>1 and len(rson.son)>1):
            pass
        elif(len(lson.son)>1):
            print(rson.name,lson.name)
            now = rson;
            now.son.append(lson);
        else:
            print(rson.name,lson.name)
            now = lson;
            now.son.append(rson)
    # exit(0)
    return now;
            

In [ ]:
nodes1 = [node(name=str(i),son=[]) for i in range(len(uf.tree()))]
for i,j in enumerate(uf.tree()):
    if(j!=-1):
        nodes1[j].son.append(nodes1[i])
    nodes1[i].value=embeddings[i];
    nodes1[i].subson=0
    # if(i==49):
    #     break
# show_tree(nodes1[-1]).show_fig()


In [ ]:
nodes1 = [node(name=str(i),son=[]) for i in range(len(uf.tree()))]
for i,j in enumerate(uf.tree()):
    if(j!=-1):
        nodes1[j].son.append(nodes1[i])
    nodes1[i].value=embeddings[i];
    nodes1[i].subson=0
    # if(i==49):
    #     break
# show_tree(nodes1[-1]).show_fig()

root  = search_tree(nodes1[-1],0.001)
# show_tree(root).show_fig()

In [ ]:
values = [];
fathers = [];
save_path = "./datas/72/"
def search_merge_tree(now,ids,save_path,values,fathers):
    fathers.append(ids);
    values.append(now.name);
    now_id = len(values)-1;
    for son in now.son:
        search_merge_tree(son,now_id,save_path,values,fathers)
search_merge_tree(root,-1,0,values,fathers)
np.save(save_path+"dataxy.npy",values)
np.save(save_path+"datalink.npy",fathers)

In [ ]:
datas = pd.read_csv("./datas/data2/data_cell.csv").set_index("Unnamed: 0")
datas

In [ ]:
celltype  = cell_type1 = pd.read_csv("../../../capital/docs/tutorials/BRCA_GSE114727_inDrop_CellMetainfo_table.tsv",sep="\t")
celltype

In [ ]:
adata1 = datas.copy()
adata1.loc[list(celltype['Cell']), 'Celltype'] = list(celltype['Celltype (minor-lineage)'])
adata1

In [ ]:
a=[1,2]
a.index(2)

In [ ]:
from scipy.spatial import KDTree
ans_value = []
ans_label = []
true_label = [];
now_labels = adata1['Celltype'].tolist();
c= -73
r= 8
now_label = 0;
now =  datas.values;
while len(now) != 0:
    rnd = np.random.randint(now.shape[0], size=1);
    rand_choice = now[rnd, :].reshape(-1)
    tree = KDTree(now);
    indices = tree.query_ball_point(rand_choice,r)
    points_within_k = now[indices]
    now = now.tolist();
    if(len(points_within_k)-1 < c):
        for i in points_within_k:
            ans_value.append(i.tolist())
            ans_label.append(now_label);
            now_label +=1
            now.remove(i.tolist());
            true_label.append(now_labels[i])
    else:
        for i in points_within_k:
            ans_value.append(i.tolist())
            ans_label.append(now_label);
            index = now.index(i.tolist());
            true_label.append(now_labels[index]);
            now.remove(now[index]);            
            now_labels.remove(now_labels[index])
        now_label+=1;
    # for i in range(len(points_within_k)):
        
    now = np.array(now);
    print("remain length: {}".format(len(now)))
    # for i in points_within_k:
    #     if(i == rand_choice).all():
    #         continue;
        
    # now.remove(rand_choice)
    # now.tolist().remove(rand_choice.tolist())
    # now = np.array(now)
    # break
# rand_choice == now
np.save('./datas/try/c_{}_r_{}_label.npy'.format(c,r),np.array(ans_label))
np.save('./datas/try/c_{}_r_{}_value.npy'.format(c,r),np.array(ans_value))
np.save('./datas/try/c_{}_r_{}_true.npy'.format(c,r),np.array(true_label))



In [ ]:
now_labels

In [ ]:
v = pd.DataFrame(ans_value)
v['label'] = ans_label
ann1  = v.groupby("label").mean()
ann1

In [ ]:
v1 = pd.DataFrame(true_label)
v1['label'] = ans_label
meta1 = v1.groupby("label").max()
meta1

In [ ]:
(v1.groupby("label").describe()[0]['count'] - v1.groupby("label").describe()[0]['freq']).sum() / total

In [ ]:
ann1.to_csv('./datas/73/datas.data',header=None)
ann1.to_csv('./datas/73/datas.csv')

In [ ]:
x, y_true, similarities = load_data('./datas/73/datas.data',1,514,0)
data_path = './datas/73/datas.data'
start = 1
end =514
lable = 0
model_path = None;
save_path = './datas/73/'
epoches=10
c=-1
np.random.seed(1234)
torch.manual_seed(1234)
# x, y_true, similarities = load_data('../../../cityu/HypHC/data/4_8/4_8.data',start,end,lable)
# x, y_true, similarities = load_data(data_path,start,end,lable)
print("{} length:{}".format(data_path,len(y_true)));
dataset = HCDataset(x, y_true, similarities, num_samples=50000)
dataloader = data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)
if(model_path ==None):
    model = HypHC(dataset.n_nodes, 2, 5e-2, 5e-2 ,0.999)
    model.to("cpu")
    Optimizer = getattr(optim, 'RAdam')
    optimizer = Optimizer(model.parameters(),0.0005)
    train(model,dataloader,optimizer,similarities,epoches);
    torch.save(model,save_path+'model.pth');
else:
    params = torch.load((model_path), map_location=torch.device('cpu'))
    model.load_state_dict(params, strict=False)
model.eval()
sim_fn = lambda x, y: torch.sum(x * y, dim=-1)
n=len(x);
d = model.normalize_embeddings(model.embeddings.weight.data)
d = project(d).detach().cpu()
ijs = sl_np_mst_ij(d,sim_fn)
uf = UnionFind(n,d,c)
uf.merge(ijs)
count=0


In [ ]:
tree = nx.DiGraph()
for i, j in enumerate(uf.tree()[:-1]):
    if(j!=-1):
        tree.add_edge(j, i)
        
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111)
circle = plt.Circle((0, 0), 20.0, color='r', alpha=0.1)
ax.add_artist(circle)

n = len(d)
embeddings = np.array(uf.pos)
# for i in range(len(embeddings)):
#     if(np.isnan(embeddings[i][0]) or embeddings[i][0] + embeddings[i][1] == 0):
#         embeddings[i] = embeddings[i-1];
#     pass;
# print(embeddings)
where_are_NaNs = np.isnan(embeddings)
embeddings[where_are_NaNs] = 0
vis = uf.vis
# xlist=[]
# ylist=[]
# labels=[]

# for i in range(n):
#     if(uf.vis[i]!=1):
#         xlist.append(embeddings[i,0]*20);
#         ylist.append(embeddings[i,1]*20);
#         labels.append(y_true[i]);
# # colors = get_colors(labels, 1234)
colors = get_colors(y_true, 1234)

# ax.scatter(xlist, ylist, c=colors, s=50, alpha=0.6)

ax.scatter(embeddings[:n, 0]*20, embeddings[:n, 1]*20, c=colors, s=50, alpha=0.6)

for i in range(len(embeddings)):
    if(i<n):
        continue;
    if(uf.mer[i]!=-1):
        # print(i,'pass')
        pass;
    else:
        ax.scatter(embeddings[i][0]*20,embeddings[i][1]*20,color='black',s=20,alpha=0.7)

    

for n1, n2 in tree.edges():
#     if(uf.vis[n1]== 1 or uf.vis[n2]== 1 ):
# #         print("1")
#         pass
#     else:
#         x1 = embeddings[n1]
#         x2 = embeddings[n2]
# #         if(n1>=n):
# #             if(uf.mer[n1]!=-1):
# #                 pass;
# #             else:
# #                 count+=1;
# #         if(n2>n):
# #             if(uf.mer[n1]!=-1):
# # # 
# #                 pass;
# #             else:
# # #                 ax.scatter(x2[0]*20,x2[1]*20,color='black',s=20,alpha=0.7)
# #                 count+=1
    
    
#         plot_geodesic(x1, x2, ax)
# #         print(x1,x2,n1,n2)
# #         break
#         pass

#     ax.axis("off")
    x1 = embeddings[n1];
    x2 = embeddings[n2];
    plot_geodesic(x1,x2,ax)

In [ ]:
from alignment import node
from alignment import show_tree
embeddings = np.array(uf.pos)


In [ ]:
def search_tree(now,c,merge_list):
    
    if(len(now.son) != 2):
        return now;
    lson = search_tree(now.son[0],c,merge_list);
    now.son[0] = lson;
    rson = search_tree(now.son[1],c,merge_list);
    now.son[1] = rson
    # lson = now.son[0]
    # rson = now.son[1]
    if(np.linalg.norm(lson.value-rson.value)<=c):
        if(len(lson.son)>1 and len(rson.son)>1):
            pass
        elif(len(lson.son)>1):
            merge_list.append((rson.name,lson.name))
            print(rson.name,lson.name)
            now = rson;
            now.son.append(lson);
        else:
            merge_list.append((rson.name,lson.name))
            print(rson.name,lson.name)
            now = lson;
            now.son.append(rson)
    # exit(0)
    return now;
            

In [ ]:
nodes1 = [node(name=str(i),son=[]) for i in range(len(uf.tree()))]
for i,j in enumerate(uf.tree()):
    if(j!=-1):
        nodes1[j].son.append(nodes1[i])
    nodes1[i].value=embeddings[i];
    nodes1[i].subson=0
    # if(i==49):
    #     break


In [ ]:
nodes1 = [node(name=str(i),son=[]) for i in range(len(uf.tree()))]
for i,j in enumerate(uf.tree()):
    if(j!=-1):
        nodes1[j].son.append(nodes1[i])
    nodes1[i].value=embeddings[i];
    nodes1[i].subson=0
    # if(i==49):
    #     break
# show_tree(nodes1[-1]).show_fig()
ans_list = []
root  = search_tree(nodes1[-1],0.001,ans_list)
# show_tree(root).show_fig()

In [ ]:
union =0
inter=0
for j in range(len(ans_list)):
    
    l = ans_list[j][0]
    l = list(adata1.obs[adata1.obs['leiden']==l]['celltype'])
    r = ans_list[j][1]
    r = list(adata1.obs[adata1.obs['leiden']==r]['celltype'])
    inter += len(l)+len(r);
    for i in l:
        if(r.__contains__(i)):
            union +=1;
            r.remove(i)

2*union / inter

In [ ]:
ann1

In [ ]:
meta1

In [ ]:
values = [];
fathers = [];
save_path = "./datas/73/"
def search_merge_tree(now,ids,save_path,values,fathers):
    fathers.append(ids);
    values.append(now.name);
    now_id = len(values)-1;
    for son in now.son:
        search_merge_tree(son,now_id,save_path,values,fathers)
search_merge_tree(root,-1,0,values,fathers)
np.save(save_path+"dataxy.npy",values)
np.save(save_path+"datalink.npy",fathers)